In [44]:
import pandas as pd
import os
import numpy as np
import chardet
import json
from bs4 import BeautifulSoup
from sklearn.linear_model import LogisticRegression
from category_encoders.target_encoder import TargetEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import GradientBoostingClassifier
import requests
dir = os.getcwd()

data_train1=dir+'/data/data_train_1/'
data_train2=dir+'/data/data_train_2/'
data_train3=dir+'/data/datasource6/'
test = dir+'/data/transformed_data/data_test/test/'
df_test = pd.read_csv('test_features.csv')

In [49]:
# Test Features Merging
'''dflist = []
for file in os.listdir(test):
    dflist.append(pd.read_excel(test+file,sheet_name=0).merge(pd.read_excel(test+file,sheet_name=1),on='id'))
test_merged = pd.concat(dflist)
test_merged.to_csv('test_features.csv')
'''

#df_test = pd.read_csv('test_features.csv')
'''apis = pd.read_csv('api_data.csv')
df_train = pd.read_csv('train_dataset_merged.csv')
df_train = df_train.replace('WEBSITE', np.nan)
df_train = df_train.set_index('id').combine_first(webdata.set_index('id'))
df_train = df_train.replace('API', np.nan)
df_train = df_train.combine_first(apis.set_index('id'))
'''
df_train = pd.read_csv('train_dataset_webdata_api.csv')
df_train = df_train.drop(['id',
                          'Unnamed: 0',
               'Unnamed: 0_x',
               'Unnamed: 0_y',
               'test strings (delete please)',
               'useless values (delete please)',
               'ldsa_team_wishes_you',
                          'countrycode',
                          'storeid',
                          'ddpsubcategory','country','countryoforigin','designer','orderdate_gmt','orderportalid','productid'
              ],axis=1)

In [54]:
# Data
# df_train['country']=pd.Categorical(df_train['country'])
# df_train['countryoforigin'] = pd.Categorical(df_train['countryoforigin'])
df_train['brand'] = pd.Categorical(df_train['brand'])
df_train['issale'] = np.where(df_train['issale']=='Yes',1,0)
df_train

,brand,category_1stlevel,ddprate,freereturn,hasitemsonbag,hasusedwishlist,isreseller,issale,isusingmultipledevices,isvip,...,region,returned,shipper,shiptypeid,size,style,tierafterorder,tierbeforeorder,userfraudstatus,userid
0,50.0,Bags,0.0000,1.0,No,Yes,No,0,No,Not VIP,...,3.0,0,2.0,2.0,17.0,4137.0,T4,NaN,3.0,2950.0
1,640.0,Clothing,0.0000,1.0,Yes,Yes,Yes,1,Yes,Not VIP,...,4.0,0,2.0,2.0,13.0,14472.0,VIP,VIP,6.0,36969.0
2,36.0,Shoes,5.0083,1.0,Yes,No,No,0,No,Not VIP,...,1.0,0,2.0,2.0,549.0,11696.0,T3,NaN,1.0,106588.0
3,279.0,Baby Boy Accessories,0.0000,1.0,Yes,Yes,No,1,No,VIP,...,2.0,0,2.0,2.0,138.0,34498.0,T2,T2,3.0,174306.0
4,17.0,Clothing,0.0000,1.0,No,Yes,No,1,Yes,Not VIP,...,3.0,0,2.0,2.0,135.0,66854.0,T4,NaN,3.0,87358.0
5,270.0,Jewellery,10.6755,1.0,No,Yes,No,0,No,Not VIP,...,4.0,0,2.0,2.0,17.0,85265.0,T2,T2,4.0,115742.0
6,87.0,Clothing,0.0000,1.0,Yes,Yes,Yes,0,Yes,VIP New,...,3.0,0,2.0,2.0,27.0,1425.0,VIP,VIP,6.0,28808.0
7,60.0,Clothing,0.0000,1.0,Yes,Yes,No,0,No,Not VIP,...,4.0,0,4.0,2.0,14.0,130964.0,NaN,NaN,3.0,218845.0
8,388.0,Clothing,35.7236,1.0,Yes,No,No,1,No,Not VIP,...,4.0,0,2.0,2.0,181.0,50655.0,T4,NaN,3.0,54051.0
9,106.0,Clothing,46.0425,1.0,No,No,No,1,No,Not VIP,...,3.0,0,2.0,2.0,69.0,46208.0,T4,T4,4.0,15912.0


In [16]:
'''# CSVs from datasource_6

csvlist = []
for file in os.listdir(data_train3):
    if('.csv' in file):
        csvlist.append(pd.read_csv(data_train3+file,engine='python'))
csvlist = pd.concat(csvlist)
csvlist.to_csv('csvlist.csv')


csvlist = pd.read_csv('csvlist.csv')

# 1st Folder
a = pd.read_csv(data_train1+'datasource1.csv', encoding='latin1')
b = pd.read_csv(data_train1+'datasource3.csv', encoding='latin1',sep='|')
c = pd.read_json(data_train1+'datasource2.json', orient='split').reset_index()
c.columns=['id','shipper', 'productid', 'isreseller', 'issale', 'category_1stlevel']

# 2nd Folder
d = pd.read_json(data_train2+'datasource5.csv').reset_index()
d = d.rename(columns={"index": "id"})
e = pd.read_csv(data_train2+'datasource7.csv')
# .html
html = pd.read_csv(data_train2+'datasource4.csv')

# WebData
webdata = pd.read_csv(data_train1+'web_data.csv')

# Targets
targets = pd.read_csv('sql_output_targets.csv')[['id','returned']]

# Train Data Set
df_train = a.merge(c,on='id').merge(b,on='id').merge(d,on='id').merge(e,on='id').merge(html,on='id').merge(csvlist,on='id').merge(targets,on='id')

'''

'# CSVs from datasource_6\n\ncsvlist = []\nfor file in os.listdir(data_train3):\n    if(\'.csv\' in file):\n        csvlist.append(pd.read_csv(data_train3+file,engine=\'python\'))\ncsvlist = pd.concat(csvlist)\ncsvlist.to_csv(\'csvlist.csv\')\n\n\ncsvlist = pd.read_csv(\'csvlist.csv\')\n\n# 1st Folder\na = pd.read_csv(data_train1+\'datasource1.csv\', encoding=\'latin1\')\nb = pd.read_csv(data_train1+\'datasource3.csv\', encoding=\'latin1\',sep=\'|\')\nc = pd.read_json(data_train1+\'datasource2.json\', orient=\'split\').reset_index()\nc.columns=[\'id\',\'shipper\', \'productid\', \'isreseller\', \'issale\', \'category_1stlevel\']\n\n# 2nd Folder\nd = pd.read_json(data_train2+\'datasource5.csv\').reset_index()\nd = d.rename(columns={"index": "id"})\ne = pd.read_csv(data_train2+\'datasource7.csv\')\n# .html\nhtml = pd.read_csv(data_train2+\'datasource4.csv\')\n\n# WebData\nwebdata = pd.read_csv(data_train1+\'web_data.csv\')\n\n# Targets\ntargets = pd.read_csv(\'sql_output_targets.csv\')[[

In [ ]:
# Counter

# Features

In [ ]:
# Category 1st Level
df_train['category_1stlevel'].value_counts().head(10)

In [ ]:
# Brand
df_train['brand'].value_counts().head(10)

In [ ]:
# Country vs Country Code
# Removed Country Code, kept Country
df_train['country'].value_counts().head(10)

In [ ]:
#df_train[['brand','returned']].groupby('brand').agg({'returned':[np.sum, 'count']})

In [8]:
# Target Encoder
df_baseline = df_train.copy(deep=True)

x_baseline = df_baseline.loc[:,['freereturn','userfraudstatus','issale']]
y_baseline = df_baseline.loc[:,'returned']

# TTSplit
X_train, X_test, y_train, y_test = train_test_split(x_baseline,y_baseline, test_size = 0.33, random_state = 42 )

X_train['freereturn'] = pd.Categorical(X_train['freereturn'])
X_train['userfraudstatus'] = pd.Categorical(X_train['userfraudstatus'])
te = TargetEncoder(min_samples_leaf = 100)
te.fit(X_train,y_train)

/Users/cc/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/Users/cc/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


TargetEncoder(cols=['freereturn', 'userfraudstatus'], drop_invariant=False,
              handle_missing='value', handle_unknown='value',
              min_samples_leaf=100, return_df=True, smoothing=1.0, verbose=0)

In [14]:
# Baseline
lr = LogisticRegression(random_state = 42)
lr.fit(X_train,y_train)
lr.predict_proba(te.transform(X_test))
roc_auc_score(y_test,lr.predict_proba(te.transform(X_test))[:,1])

/Users/cc/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.5301074142669557

# Gradient Boosting Classifier

In [47]:
gbc = GradientBoostingClassifier(random_state = 42)
gbc.fit(X_train,y_train)
roc_auc_score(y_test,gbc.predict_proba(te.transform(X_test))[:,1])

0.5

In [48]:
importance = pd.DataFrame({'features': X_train.columns,
                           'importance': gbc.feature_importances_
                          }).sort_values(by = 'importance', ascending=False)
importance

,features,importance
1,userfraudstatus,0.689995
0,freereturn,0.310005


In [20]:
df_test[['id','freereturn','userfraudstatus']]
ids = df_test[['id']]

In [22]:
features = df_test[['freereturn','userfraudstatus']]

In [34]:
lr.predict_proba(te.transform(features))[:,1]

array([0.05865343, 0.05863444, 0.05863444, ..., 0.05865763, 0.05865763,
       0.05865763])

In [36]:
submission = df_test[['id']]
submission['returned']=lr.predict_proba(te.transform(features))[:,1]
submission.to_csv('SUBMISSION_01.csv',index=False)

/Users/cc/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
